In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import *
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from IPython.display import display
from PIL import Image
import time

t0 = time.time()

    # dimensions of our images.
img_width, img_height = 150, 150

train1_data_dir = 'data/train1'
train2_data_dir = 'data/train2'
train3_data_dir = 'data/train3'
val1_data_dir = 'data/val1'
val2_data_dir = 'data/val2'
val3_data_dir = 'data/val3'
nb_train_samples = 999
nb_validation_samples = 501
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


'''
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
'''

model1 = Sequential()
model1.add(Conv2D(32, (3, 3), input_shape=input_shape))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model2 = Sequential()
model2.add(Conv2D(32, (3, 3), input_shape=input_shape))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))

model3 = Sequential()
model3.add(Conv2D(32, (3, 3), input_shape=input_shape))
model3.add(Activation('relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

mergedOut = Add()([model1.output,model2.output,model3.output])

mergedOut = Flatten()(mergedOut)    
mergedOut = Dense(64, activation='relu')(mergedOut)
mergedOut = Dropout(.5)(mergedOut)
mergedOut = Dense(2, activation='sigmoid')(mergedOut)

model = Model([model1.input,model2.input,model3.input], mergedOut)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

def generate_generator_multiple(generator,dir1, dir2, dir3, batch_size, img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    genX2 = generator.flow_from_directory(dir2,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    genX3 = generator.flow_from_directory(dir3,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)
    
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            yield [X1i[0], X2i[0], X3i[0]], X3i[1]  #Yield both images and their mutual label
            
            
train_generator=generate_generator_multiple(generator=train_datagen,
                                           dir1=train1_data_dir,
                                           dir2=train2_data_dir,
                                           dir3=train3_data_dir,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_height)       


validation_generator=generate_generator_multiple(generator=test_datagen,
                                          dir1=val1_data_dir,
                                          dir2=val2_data_dir,
                                          dir3=val3_data_dir,
                                          batch_size=batch_size,
                                          img_height=img_height,
                                          img_width=img_height) 

'''
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
'''



model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('para_try.h5')

t1 = time.time()

total = t1-t0
    
print("Estimated time:",total,"s")

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Epoch 1/50
Found 666 images belonging to 2 classes.
Found 666 images belonging to 2 classes.
Found 666 images belonging to 2 classes.
61/62 [============================>.] - ETA: 0s - loss: 7.1725 - acc: 0.5441Found 334 images belonging to 2 classes.
Found 334 images belonging to 2 classes.
Found 334 images belonging to 2 classes.
62/62 [==============================] - 31s 503ms/step - loss: 7.1868 - acc: 0.5434 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/50
62/62 [==============================] - 26s 414ms/step - loss: 8.0671 - acc: 0.4995 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/50
62/62 [==============================] - 26s 420ms/step - loss: 4.3419 - acc: 0.5418 - val_loss: 0.7443 - val_acc: 0.6296
Epoch 4/50
62/62 [==============================] - 26s 415ms/step - loss: 0.7061 - acc: 0.5753 - val_loss: 1.2719 - val_acc: 0.6280
Epoch 5/50
62/62 [==============================] - 26s 412ms/step - loss: 0.6136 - acc: 0.6737 - val_loss: 6.0775 - val_acc: 0.6113
Epoch 6/50
6